In [ ]:
#TASK 1 - Please download dataset or generate it using the attached script dataset_generator.py
import pandas as pd
from tqdm import tqdm
import csv
import random
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

random.seed(1)

letters = string.ascii_lowercase
letters_upper = string.ascii_uppercase
for _i in range(0, 10):
    letters += letters

for _i in range(0, 10):
    letters += letters_upper


def random_string(stringLength=10):
    """Generate a random string of fixed length """
    return ''.join(random.sample(letters, stringLength))


print("Products between {} and {}".format(1, 100000))
product_ids = [x for x in range(1, 100000)]
dates = ['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10']
seller_ids = [x for x in range(1, 10)]


#   Generate products
products = [[0, "product_0", 22]]
for p in tqdm(product_ids):
    products.append([p, "product_{}".format(p), random.randint(1, 150)])
#   Save dataframe
df = pd.DataFrame(products)
df.columns = ["product_id", "product_name", "price"]
df.to_csv("products.csv", index=False)
del df
del products

#   Generate sellers
sellers = [[0, "seller_0", 100000]]
for s in tqdm(seller_ids):
    sellers.append([s, "seller_{}".format(s), random.randint(0, 100000)])
#   Save dataframe
df = pd.DataFrame(sellers)
df.columns = ["seller_id", "seller_name", "daily_target"]
df.to_csv("sellers.csv", index=False)

#   Generate sales
total_rows = 100000
prod_zero = int(total_rows * 0.95)
prod_others = total_rows - prod_zero + 1
df_array = [["order_id", "product_id", "seller_id", "date", "num_pieces_sold", "bill_raw_text"]]
with open('sales.csv', 'w', newline='') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerows(df_array)

order_id = 0
for i in tqdm(range(0, 40)):
    df_array = []

    for i in range(0, prod_zero):
        order_id += 1
        df_array.append([order_id, 0, 0, random.choice(dates), random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

    df_array = []
    for i in range(0, prod_others):
        order_id += 1
        df_array.append(
            [order_id, r6andom.choice(product_ids), random.choice(seller_ids), random.choice(dates),
             random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

print("Done")

In [ ]:
#-------------------------TASK 1----------------------------------------------
#importing SparkSession class
from pyspark.sql import SparkSession

In [ ]:
#assigning config to spark variable
spark=SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option","some-value") \
.getOrCreate()

In [ ]:
#creating sparkcontext
sc=spark.sparkContext

In [ ]:
#2.1. Please upload data from sellers.csv into sellers_rdd variable. Use the textFile method with map function: 
sellers_rdd=sc.textFile("/home/jovyan/work/sellers.csv").map(lambda x:x.split(","))

In [ ]:
#2.3. Save the first element of sellers_rdd into sellers_rdd_header variable
sellers_rdd_header=sellers_rdd.top(1)

In [ ]:
#2.4. Filter the sellers_rdd excluding the sellers_rdd_header value, and reassign it to sellers_rdd. 
sellers_rdd=sellers_rdd.filter(lambda x:x not in sellers_rdd_header)

In [ ]:
sellers_rdd.collect()

In [ ]:
#2.	Create a DataFrame variable sellers_df_test using toDF() function.
sellers_df_test=sellers_rdd.toDF(schema=sellers_rdd_header[0])

In [ ]:
#3.	Run the following code in order to change data types 
sellers_rdd = sellers_rdd.map(lambda x: (int(x[0]),x[1],int(x[2])))

In [ ]:
sellers_rdd.collect()

In [ ]:
#4.	Recreate a DataFrame variable sellers_df using createDataFrame() function. For this task you need to create a schema. 
#Review the dataset in order to choose correct data types.

In [ ]:
#create a schema
from pyspark.sql.types import *
sellers_rdd_head=StructType([StructField('seller_id',IntegerType(),True),
StructField('seller_name',StringType(),True),
StructField('daily_target',IntegerType(),True)])

In [ ]:
#Recreate a DataFrame variable sellers_df using createDataFrame() function.
sellers_df=spark.createDataFrame(sellers_rdd,schema=sellers_rdd_head)

In [ ]:
sellers

In [ ]:
#Review the dataset in order to choose correct data types.
sellers_df.printSchema()

In [ ]:
#5.	Create a Pandas dataframe sellers_df_pd  from Spark dataframe.
sellers_df_Pandas=sellers_df.toPandas()

In [ ]:
#6.	Take first 5 rows of sellers_df
sellers_df.head(5)

In [ ]:
#7.	Return a schema and dtypes of sellers_df.

In [ ]:
sellers_df.schema

In [ ]:
sellers_df.dtypes

In [ ]:
#1.	Upload data from products.csv to products_df Dataframe variable and sales.csv to sales_df using the spark.read.csv() function.
#Please take a look on the header parameter while data loading.

In [ ]:
products_df=spark.read.csv("/home/jovyan/work/products.csv",header=True)

In [ ]:
products_df.show()

In [ ]:
sales_df=spark.read.csv("/home/jovyan/work/sales.csv",header=True)

In [ ]:
sales_df.show()

In [ ]:
#3.	Show the summary statistic of products_df
products_df.describe().show()

In [ ]:
#4.	Print the physical and logical plans of products_df
products_df.explain()

In [ ]:
#5.	Count a number of distinct rows of products_df
products_df.distinct().count()

In [ ]:
#6.	Transform a dataframe to RDD products_rdd variable using products_df.rdd method and print a number of partitions
products_rdd=products_df.rdd

In [ ]:
products_rdd.getNumPartitions()

In [ ]:
#7.	Filter products_df where price > 100 and save a dataframe to the products_df_more_than_100.csv file. 
products_csv=products_df.filter(products_df["price"]>100)

In [ ]:
products_csv.write.csv("/home/jovyan/work/products_df_more_than_100",mode="overwrite",header=True)

In [ ]:
import re

In [ ]:
m = re.search('[^\s\\]',str(products_df))

In [ ]:
products_df.toJSON().collect()

In [259]:
#8.	Save products_df to JSON and Parquet files.
#8.	Save products_df to JSON and Parquet files.
products_df.write.json("/home/jovyan/work/products_df_json",mode="overwrite")


In [261]:
#8.	Save products_df to JSON and Parquet files.
products_df.write.parquet("/home/jovyan/work/products_df_parquet",mode="overwrite",partitionBy='price')

In [ ]:
#9.	Upload the JSON file from the step above to the products_df_from_json Dataframe variable and show the content of it.
products_df_from_json=spark.read.json("/home/jovyan/work/products_df_json")

In [ ]:
products_df_from_json.show()